In [1]:
%run ../src/ipyautoui/__init__.py
%load_ext lab_black

In [19]:
print(" not yet implemented".upper())

 NOT YET IMPLEMENTED


# Validation - NOTE:  NOT YET IMPLEMENTED

- jsonschema validation occurs on-change by default (TODO: implement!)
    - review https://github.com/pydantic/pydantic-core
- the top-level pydantic model can also be validated on request (slows things down...?)
    - possible to implement recursive validation with pydantic? 

In [1]:
from jsonschema import validate, Draft202012Validator, ErrorTree

# A sample schema, like what we'd get from json.load()
schema = {
    "type" : "object",
    "properties" : {
        "price" : {"type" : "number"},
        "name" : {"type" : "string"},
    },
}

# If no exception is raised by validate(), the instance is valid.
validate(instance={"name" : "Eggs", "price" : 34.99}, schema=schema)

validate(
    instance={"name" : "Eggs", "price" : "Invalid"}, schema=schema,
)      

ValidationError: 'Invalid' is not of type 'number'

Failed validating 'type' in schema['properties']['price']:
    {'type': 'number'}

On instance['price']:
    'Invalid'

In [4]:
schema = {
    "items": {
        "anyOf": [{"type": "string", "maxLength": 2}, {"type": "integer", "minimum": 5}]
    }
}
instance = [{}, 3, "foo"]
v = Draft202012Validator(schema)
errors = sorted(v.iter_errors(instance), key=lambda e: e.path)
errors

[<ValidationError: '{} is not valid under any of the given schemas'>,
 <ValidationError: '3 is not valid under any of the given schemas'>,
 <ValidationError: "'foo' is not valid under any of the given schemas">]